In [1]:
import requests
import pandas as pd
import json

## Variables de entorno, colocar en un .ENV

In [ ]:
USDA_API_KEY = "12F1F802-DE5D-3990-B649-6CC02E679BFC"

## Obtener la data de nwcc (national water and climate center) de las 200 antendas de la red SCAN. Una red centrada en lo puntos de cultivos de USA.

In [ ]:
# URL del endpoint
url = "https://wcc.sc.egov.usda.gov/awdbRestApi/services/v1/stations"

NETWORK = "SCAN"

try:
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código 200)
    if response.status_code == 200:
        data = response.json()
        stations_df = pd.DataFrame(data)
        stations_df = stations_df[stations_df['networkCode'] == 'SCAN'] 
        
        #print(data)
    else:
        print(f"Error en la solicitud: {response.status_code} - {response.text}")

except Exception as e:
    print(f"Ocurrió un error: {e}")


In [4]:
stations_df.to_csv('stations.csv')

## Obtencion de los datos de rendimiento de maiz desde el año 2013 a 2020 en Iowa 

In [3]:
# Definir la URL de la API
url = (
    f"https://quickstats.nass.usda.gov/api/api_GET/?key={USDA_API_KEY}"
    "&source_desc=SURVEY"
    "&sector_desc=CROPS"
    "&group_desc=FIELD CROPS"
    "&commodity_desc=CORN"
    "&statisticcat_desc=YIELD"
    "&year__GE=2013"
    "&year__LT=2020"
    "&agg_level_desc=STATE"
    "&unit_desc=BU / ACRE"
    "&state_alpha=IA"
)

# Realizar la solicitud GET a la API
response = requests.get(url)

# Verificar el estado de la respuesta
if response.status_code == 200:
    data = response.json()
    corn_yield_df = pd.DataFrame(data['data'])
    # Guardar los datos en un archivo JSON
    print("Datos guardados en corn yield df")
else:
    print(f"Error en la solicitud: {response.status_code}")


Datos guardados en corn yield df


## Obtener los datos de una estacion en especifico

In [36]:

url_awdb = (
    "https://wcc.sc.egov.usda.gov/awdbRestApi/services/v1/data"
    "?stationTriplets=2057:AL:SCAN"
    "&elements=TAVG, TMAX, TMIN"
    "&duration=DAILY"
    "&beginDate=2025-02-19"
    "&endDate=2025-02-25"
    "&returnFlags=true"
    "&returnOriginalValues=true"
    "&returnSuspectData=true"
)
# Realizar la solicitud GET a la API AWDB
response_awdb = requests.get(url_awdb)
if response_awdb.status_code == 200:
    data_awdb = response_awdb.json()
    with open("temperature_AL.json", "w") as f:
        json.dump(data_awdb, f, indent=4)
    print("Datos de temperatura guardados en temperature_AL.json")
    values = data_awdb[0]['data'][0]['values']
    df = pd.DataFrame(values)
    df = df[['date', 'value']]
    df.rename(columns={'value': 'temperature_F'}, inplace=True)
    print(df)
else:
    print(f"Error en la solicitud AWDB: {response_awdb.status_code}")


Datos de temperatura guardados en temperature_AL.json
         date  temperature_F
0  2025-02-19           27.0
1  2025-02-20           20.5
2  2025-02-21           25.2
3  2025-02-22           30.6
4  2025-02-23           35.8
5  2025-02-24           44.6
